# <span style="color:#4682B4">NCBI Taxonomy Data Profile</span>
---

The NCBI Taxonomy is a reference database that contains the names and hierarchically-arranged phylogenetic classifications of organisms created in 1991. The NCBI Taxonomy database is organized in a tree for a hierarchical data structure, where each <span style="color:#4682B4">**_node_**</span> of the tree represents a <span style="color:#4682B4">**_taxon_**</span> and each entry has a primary name, secondary names, and a unique taxonomic identifier. The NCBI Taxonomy database is critical to link nucleotides and protein sequences from the International Nucleotide Sequence Database Collaboration (INSDC) and other biological databases which rely on data from INSDC. These linkages can be made using either the organism name or taxonomic ID.

The database is provided remotely by the National Center for Biotechnology Information (NCBI). INSDC partners send any requests for new names to NCBI Taxonomy curators before data is released. The database incorporates phylogenetic and taxonomic knowledge from published literature, web-based databases, and the advice of sequence submitters and outside taxonomy experts.

### <span style="color:#4682B4">Why NCBI Taxonomy</span>
We chose to use the NCBI Taxonomy database because it is the <span style="color:#4682B4">**_sole source for taxonomic classification_**</span> for the INSDC and forms the backbone for many other resources at the NCBI. The NCBI Taxonomy database contains formal and informal organism names and classifications for every sequence in INSDC's datasets (more than 160,000 organisms); these associations between pathogen name and genetic and genomic data are foundational for public health intelligence efforts. The inclusion of informal names also allows us to link pathogens (and their corresponding genetic information) to case reports and other non-traditional data sources which may use names which exist outside of the codes of nomenclature (i.e. "COVID-19" instead of SARS-CoV-2). Additionally, more than 150 external partners maintain links to the NCBI Taxonomy database, with specialty datasets of their own.

### <span style="color:#4682B4">Accessing the NCBI Taxonomy database</span>

There are three methods for accessing the NCBI Taxonomy. First is the NCBI Taxonomy Browser, which is a web-page that allows users to search for organisms, visualize the hierarchy at custom levels of classification, and summarize organism information, such as lineage, in a taxon-specific page. 

In contrast to the Taxonomy Browser, Entrez supports Boolean queries and common search fields across all NCBI databases. There are also several public APIs that allow programmatic access to the Entrez databases; <span style="color:#4682B4">**_we used E-utilities,_**</span> a suite of server-side programs that accept a fixed URL syntax for search, link, and retrieval. 

A third option is to download the complete database as a full text taxdump file (in .dmp format), which is updated every hour on the site. 

This notebook examines the information stored within the NCBI Taxonomy and the quality of metadata that is retrievable via publicly available APIs.

### <span style="color:gray">_Connection to the database_</span>
---

In [1]:
import sys

print("Checking python executable path (make sure it's the right virtualenv)")
print(sys.executable)
# append the path of the parent directory
sys.path.append("..")


Checking python executable path (make sure it's the right virtualenv)
/Users/haileyrobertson/Documents/GitHub/kr2-graph/build_graph_2/notebooks/env/bin/python


In [2]:
import ncbi


In [3]:
print(ncbi.id_search('Covid 19'))


2022-05-03 14:05:20.183 | INFO     | ncbi.id_search:id_search:7 - Searching ncbi for term Covid 19


2697049
